In [41]:
import re
import pandas as pd
import numpy as np
from IPython.display import display, HTML

test ="""<?xml version="1.0" encoding="utf-8"?>
<session sessioncode="4949274597">
 <general>
  <client_version>22.9.1.11</client_version>
  <mode>real</mode>
  <gametype>Omaha PL 0,01€/0,02€</gametype>
  <tablename>Sasolburg, 560235858</tablename>
  <tablecurrency>EUR</tablecurrency>
  <smallblind>0,01€</smallblind>
  <bigblind>0,02€</bigblind>
  <duration>00:31:53</duration>
  <gamecount>45</gamecount>
  <startdate>2022-11-03 20:42:01</startdate>
  <currency>EUR</currency>
  <nickname>jedeop</nickname>
  <bets>11,42€</bets>
  <wins>7,26€</wins>
  <chipsin>1,98€</chipsin>
  <chipsout>4,83€</chipsout>
  <statuspoints>0,03</statuspoints>
  <awardpoints>0,00</awardpoints>
  <ipoints>0,03</ipoints>
  <tablesize>6</tablesize>
 </general>
 <game gamecode="6175905121">
  <general>
   <startdate>2022-11-03 20:42:01</startdate>
   <players>
    <player win="0€" bet="0,02€" name="Player 5" dealer="0" chips="3,35€" seat="5"/>
    <player win="0,47€" bet="0,53€" name="Player 6" dealer="1" chips="0,64€" seat="6"/>
    <player win="0€" bet="0,16€" name="Player 8" dealer="0" chips="0,44€" seat="8"/>
    <player win="0€" bet="0,16€" name="Player 10" dealer="0" chips="0,60€" seat="10"/>
    <player win="0€" bet="0€" name="Player 3" dealer="0" chips="1,33€" seat="3"/>
   </players>
  </general>
  <round no="0">
   <action player="Player 8" type="1" no="1" sum="0,01€"/>
   <action player="Player 10" type="2" no="2" sum="0,02€"/>
  </round>
  <round no="1">
   <cards player="Player 3" type="Pocket">X X X X</cards>
   <action player="Player 3" type="0" no="3" sum="0€"/>
   <cards player="Player 5" type="Pocket">X X X X</cards>
   <action player="Player 5" type="3" no="4" sum="0,02€"/>
   <cards player="Player 6" type="Pocket">X X X X</cards>
   <action player="Player 6" type="3" no="5" sum="0,02€"/>
   <cards player="Player 8" type="Pocket">X X X X</cards>
   <action player="Player 8" type="3" no="6" sum="0,01€"/>
   <cards player="Player 10" type="Pocket">X X X X</cards>
   <action player="Player 10" type="4" no="7" sum="0€"/>
  </round>
  <round no="2">
   <cards type="Flop">D10 CA HQ</cards>
   <action player="Player 8" type="4" no="8" sum="0€"/>
   <action player="Player 10" type="4" no="9" sum="0€"/>
   <action player="Player 5" type="4" no="10" sum="0€"/>
   <action player="Player 6" type="5" no="11" sum="0,04€"/>
   <action player="Player 8" type="3" no="12" sum="0,04€"/>
   <action player="Player 10" type="3" no="13" sum="0,04€"/>
   <action player="Player 5" type="0" no="14" sum="0€"/>
  </round>
  <round no="3">
   <cards type="Turn">C4</cards>
   <action player="Player 8" type="4" no="15" sum="0€"/>
   <action player="Player 10" type="4" no="16" sum="0€"/>
   <action player="Player 6" type="5" no="17" sum="0,10€"/>
   <action player="Player 8" type="3" no="18" sum="0,10€"/>
   <action player="Player 10" type="3" no="19" sum="0,10€"/>
  </round>
  <round no="4">
   <cards type="River">D7</cards>
   <action player="Player 8" type="4" no="20" sum="0€"/>
   <action player="Player 10" type="4" no="21" sum="0€"/>
   <action player="Player 6" type="5" no="22" sum="0,37€"/>
   <action player="Player 8" type="0" no="23" sum="0€"/>
   <action player="Player 10" type="0" no="24" sum="0€"/>
  </round>
 </game>
"""

substitutions = {
                     'LS'  : u"\$|\xe2\x82\xac|\xe2\u201a\xac|\u20ac|\xc2\xa3|\£|RSD|",
                     'PLYR': r'(?P<PNAME>[^\"]+)',
                     'NUM' : r'(.,\d+)|(\d+)',
                    }

re_Identify = re.compile(u"""<?xml version="1.0" encoding="utf-8"?>
<session sessioncode=""",  re.MULTILINE)

m = re_Identify.search(test)
print(m)
if m != None:

    mg = m.groupdict()

    df = pd.DataFrame.from_dict(mg, orient='index')
    df = df.transpose()
    display(HTML(df.to_html()))

None


In [26]:
re_GameInfo = re.compile(r"""
           <gametype>(?P<GAME>((?P<CATEGORY>(5|7)\sCard\sStud(\sHi\-Lo)?|(Six\sPlus\s)?Holdem|Omaha(\sHi\-Lo)?)?\s?(?P<LIMIT>NL|SL|L|LZ|PL|БЛ|LP|No\slimit|Pot\slimit|Limit))|LH\s(?P<LSB>[%(NUM)s]+)(%(LS)s)?/(?P<LBB>[%(NUM)s]+)(%(LS)s)?.+?)
            (\s(%(LS)s)?(?P<SB>[%(NUM)s]+)(%(LS)s)?/(%(LS)s)?(?P<BB>[%(NUM)s]+))?(%(LS)s)?(\sAnte\s(%(LS)s)?(?P<ANTE>[%(NUM)s]+)(%(LS)s)?)?</gametype>\s+?
            <tablename>(?P<TABLE>.+)?</tablename>\s+?
            (<(tablecurrency|tournamentcurrency)>(?P<TABLECURRENCY>.*)</(tablecurrency|tournamentcurrency)>\s+?)?
            (<smallblind>.+</smallblind>\s+?)?
            (<bigblind>.+</bigblind>\s+?)?
            <duration>.+</duration>\s+?
            <gamecount>.+</gamecount>\s+?
            <startdate>.+</startdate>\s+?
            <currency>(?P<CURRENCY>.+)?</currency>\s+?
            <nickname>(?P<HERO>.+)?</nickname>
            """ % substitutions, re.MULTILINE|re.VERBOSE)

m = re_GameInfo.search(test)
if m != None:

    mg = m.groupdict()

    df = pd.DataFrame.from_dict(mg, orient='index')
    df = df.transpose()
    display(HTML(df.to_html()))

,GAME,CATEGORY,LIMIT,LSB,LBB,SB,BB,ANTE,TABLE,TABLECURRENCY,CURRENCY,HERO
0,Omaha PL,Omaha,PL,None,None,"0,01","0,02",None,"Sasolburg, 560235858",EUR,EUR,jedeop


In [4]:
re_HandInfo = re.compile(r'code="(?P<HID>[0-9]+)">\s*?<general>\s*?<startdate>(?P<DATETIME>[\.a-zA-Z-/: 0-9]+)</startdate>', re.MULTILINE)
    
m = re_HandInfo.search(test)
if m != None:

    mg = m.groupdict()

    df = pd.DataFrame.from_dict(mg, orient='index')
    df = df.transpose()
    display(HTML(df.to_html()))

,HID,DATETIME
0,6175905121,2022-11-03 20:42:01


In [5]:
re_PlayerInfo = re.compile(r'<player( (seat="(?P<SEAT>[0-9]+)"|name="%(PLYR)s"|chips="(%(LS)s)?(?P<CASH>[%(NUM)s]+)(%(LS)s)?"|dealer="(?P<BUTTONPOS>(0|1))"|win="(%(LS)s)?(?P<WIN>[%(NUM)s]+)(%(LS)s)?"|bet="(%(LS)s)?(?P<BET>[^"]+)(%(LS)s)?"|addon="\d*"|rebuy="\d*"|merge="\d*"|reg_code="[\d-]*"))+\s*/>' % substitutions, re.MULTILINE)
    
m = re_PlayerInfo.search(test)
if m != None:

    mg = m.groupdict()

    df = pd.DataFrame.from_dict(mg, orient='index')
    df = df.transpose()
    display(HTML(df.to_html()))

,SEAT,PNAME,CASH,BUTTONPOS,WIN,BET
0,5,Player 5,"3,35",0,0,"0,02€"


In [6]:
re_Board = re.compile(r'<cards( (type="(?P<STREET>Flop|Turn|River)"|player=""))+>(?P<CARDS>.+?)</cards>', re.MULTILINE)
    
m = re_Board.search(test)
if m != None:

    mg = m.groupdict()

    df = pd.DataFrame.from_dict(mg, orient='index')
    df = df.transpose()
    display(HTML(df.to_html()))

,STREET,CARDS
0,Flop,D10 CA HQ


In [7]:
re_Hero = re.compile(r'<nickname>(?P<HERO>.+)</nickname>', re.MULTILINE)
   
m = re_Hero.search(test)
if m != None:

    mg = m.groupdict()

    df = pd.DataFrame.from_dict(mg, orient='index')
    df = df.transpose()
    display(HTML(df.to_html()))

,HERO
0,jedeop


In [8]:
re_HeroCards = re.compile(r'<cards( (type="(Pocket|Second\sStreet|Third\sStreet|Fourth\sStreet|Fifth\sStreet|Sixth\sStreet|River)"|player="%(PLYR)s"))+>(?P<CARDS>.+?)</cards>' % substitutions, re.MULTILINE)
    
m = re_HeroCards.search(test)
if m != None:

    mg = m.groupdict()

    df = pd.DataFrame.from_dict(mg, orient='index')
    df = df.transpose()
    display(HTML(df.to_html()))

,PNAME,CARDS
0,Player 3,X X X X


In [47]:
test2="""<action type="0" player="Player 3" no="3" sum="0€"/>
<action type="0" player="Player 3" no="3" sum="0€"/>
"""
re_Action = re.compile(r'<action(?:\s+player=\"%(PLYR)s\"|\s+type=\"(?P<ATYPE>\d+)\"|\s+no=\"(?P<ACT>[0-9]+)\"|\s+sum=\"(?P<BET>[%(NUM)s]+)(%(LS)s)\")+/>' % substitutions, re.MULTILINE)
      
m = re_Action.search(test)
if m != None:

    mg = m.groupdict()

    df = pd.DataFrame.from_dict(mg, orient='index')
    df = df.transpose()
    display(HTML(df.to_html()))

,PNAME,ATYPE,ACT,BET
0,Player 8,1,1,"0,01"
